# 0. Installing geemap

For this work is necesary install geemap library. To do it, follow the next page:

* https://geemap.org/installation/

# 1. Filtring by target contries

In [ ]:
%load_ext pycodestyle_magic

In [ ]:
%pycodestyle_on

In [ ]:
# import libraries
import os
# import fiona
import ee
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize()

In [ ]:
# Target countries: Option 1
listCountries = ['Argentina', 'Bolivia', 'Brazil', 'Colombia', 'Ecuador',
                 'Guatemala', 'Honduras', 'Mexico', 'Peru', 'Venezuela']

# Load dataset
dataset = ee.FeatureCollection("FAO/GAUL/2015/level2")


# Function to filter by country
def filterCountry(country):
    countries = ee.FeatureCollection("FAO/GAUL/2015/level2")
    filter_ = countries.filter(ee.Filter.eq('ADM0_NAME', country))
    return filter_


# create a list using the previous function to filter the nine countries
list_filterCountries = [filterCountry(i) for i in listCountries]

# Join the nine countries
combinedCountries = list_filterCountries[0].merge(
    list_filterCountries[1]).merge(
    list_filterCountries[2]).merge(
    list_filterCountries[3]).merge(
    list_filterCountries[4]).merge(
    list_filterCountries[5]).merge(
    list_filterCountries[6]).merge(
    list_filterCountries[7]).merge(
    list_filterCountries[8]).merge(
    list_filterCountries[9])

# visualize the map
Map = geemap.Map()
tyleParams = {
    'fillColor': 'b5ffb4',
    'color': '00909F',
    'width': 1.0
}

Map.addLayer(combinedCountries, tyleParams, 'Countries', 1)

Map.addLayerControl()
Map

In [ ]:
# Filter target countries: Option 2
listCountries = ['Argentina', 'Bolivia', 'Brazil', 'Colombia', 'Ecuador',
                 'Guatemala', 'Honduras', 'Mexico', 'Peru', 'Venezuela']

# Load dataset
dataset = ee.FeatureCollection("FAO/GAUL/2015/level2")

targetCountries = dataset.filter(ee.Filter.inList(
  'ADM0_NAME', listCountries))

# Visualize map
Map = geemap.Map()
tyleParams = {
    'fillColor': 'b5ffb4',
    'color': '00909F',
    'width': 1.0
}

Map.addLayer(targetCountries, tyleParams, 'Countries', 1)

Map.addLayerControl()
Map

In [ ]:
# Calculate area in square kilometers
def areaCountries(feature):
    area = feature.geometry().area().divide(1e6)
    return feature.set('areaSqKm', area)


result2 = targetCountries.map(areaCountries)

Map.addLayer(result2, tyleParams, 'TargetCountries', 1)

# Map.addLayerControl()
Map

# 2. Global Forest Change

## 2.1 Quantifying Forest Change

### 2.1.1. Select bands

In [ ]:
# Load Global Forest Change dataset
gfc2020 = ee.Image("UMD/hansen/global_forest_change_2020_v1_8")

# selecting bands
treecover = gfc2020.select(['treecover2000'])
lossImage = gfc2020.select(['loss'])
lossYearImage = gfc2020.select(['lossyear'])
gainImange = gfc2020.select(['gain'])

In [ ]:
# Defining the pixel size
scale = lossImage.projection().nominalScale()
print('In square meters: ', scale.getInfo())

### 2.1.1.  Defining forest to thresholds of greater than 30, 50 and 70%

In [ ]:
# treecover grater than 70%
treecover70 = treecover.gte(70).updateMask(treecover.gte(70))

# treecover grather than 50%
treecover50 = treecover.gte(50).updateMask(treecover.gte(50))

# treecover grather than 30%
treecover30 = treecover.gte(30).updateMask(treecover.gte(30))

# Add tree covers
Map.addLayer(treecover70, {
    'palette': ['00FF00'],
}, 'Forest_70')

Map.addLayer(treecover50, {
    'palette': ['00FF00']
}, 'Forest_50')

Map.addLayer(treecover30, {
    'palette': ['00FF00']
}, 'Forest_30')

Map

# 3. Zonal Statistics

In [ ]:
# definiting temporal scale
consecutive_numbers = np.arange(1, 21)
years = np.arange(2001, 2021)

consecutive_numbers, years

In [ ]:
list_lossyear = [
    lossYearImage.eq(i+1).mask(lossYearImage.eq(i+1)) for i in range(
        len(consecutive_numbers))
]

In [ ]:
# create a list with names to download the zonal statistic
lossyear70 = ['lossyear70_' + str(i) for i in years]

lossyear50 = ['lossyear50_' + str(i) for i in years]

lossyear30 = ['lossyear30_' + str(i) for i in years]

In [ ]:
# Use And() method to create loss band by year according to the porcentage of

# treecover previously defined

deforestation50 = [i.And(treecover50) for i in list_lossyear]
deforestation70 = [i.And(treecover70) for i in list_lossyear]
deforestation30 = [i.And(treecover30) for i in list_lossyear]

In [ ]:
# Add layers
Map = geemap.Map(center=(4, -75), zoom=8)

Map.addLayer(list_lossyear[9], {'palette': ['000000']}, 'lossYear')
Map.addLayer(deforestation50[9], {'palette': ['00FF00']}, 'deforestation_50')
Map.addLayer(deforestation70[9], {'palette': ['FF0000']}, 'deforestation_70')
Map.addLayer(deforestation30[0], {'palette': ['FF0000']}, 'deforestation_30')
Map

## 3.1. For each loss year using treecover30%

In [ ]:
def TargetCountries30(country):
    # filter for target countries
    countryTarget = result2.filter(ee.Filter.eq(
      'ADM0_NAME', country))

    for i in range(len(consecutive_numbers)):
        out_dir = os.path.join('outputs_zonalStatistic')
        out_countries_stats = os.path.join(
            out_dir, lossyear30[i] + country + '.csv')

        geemap.zonal_statistics(
            deforestation30[i],
            countryTarget,
            out_countries_stats,
            statistics_type='SUM',
            scale=30
        )
    # -------------------------------------------------------------------------
    # Load csv layers generated in the previous step

    filename = r'outputs_zonalStatistic/'

    filenames = ['lossyear30_' + str(f) + country + '.csv' for f in years]

    df_DEF_country30 = [pd.read_csv(filename + j) for j in filenames]

    # Calculate area in ha from result of the zonal statistic
    for i in range(len(df_DEF_country30)):
        df_DEF_country30[i]['loss30_ha_'+str(years[i])] = (
            df_DEF_country30[i]['sum']*(scale**2))/10000

    # download for only one time the FeatureCollection of the countries
    # in shape format

    # This is to get a base in shp format to join csv's fiels.
    out_dir = os.path.join('outputs_zonalStatistic/countriesSHP')
    out_countries_stats = os.path.join(out_dir, country+'30.shp')

    geemap.zonal_statistics(
        deforestation30[0],
        countryTarget,
        out_countries_stats,
        statistics_type='SUM',
        scale=scale)

    # Drop unnecesary columns
    df_DEF_country30_drop = []
    for i in df_DEF_country30:
        df_DEF_country30_drop.append(i.drop([
            'sum', 'system:index', 'ADM1_CODE', 'EXP2_YEAR', 'ADM2_NAME',
            'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
            'ADM0_NAME', 'ADM0_CODE', 'Shape_Area', 'areaSqKm'
        ], axis=1))

    gpd_country_30 = gpd.read_file(
        r'outputs_zonalStatistic/countriesSHP/'+country+'30.shp')
    gpd_country_30 = gpd_country_30.drop(['sum'], axis=1)

    gpd_country_30 = gpd_country_30.set_index('ADM2_CODE')

    for i in df_DEF_country30_drop:
        gpd_country_30 = pd.concat([gpd_country_30, i.set_index('ADM2_CODE')],
                                   axis=1)

    out_dir = os.path.join('outputs_zonalStatistic')
    out_countries_stats = os.path.join(out_dir,
                                       'treecover2000_30_'+country+'.csv')

    geemap.zonal_statistics(
        treecover30,
        countryTarget,
        out_countries_stats,
        statistics_type='SUM',
        scale=scale)

    # -------------------------------------------------------------------------
    # concat treecover 30%
    df_treecover2000_country30 = pd.read_csv(
        r'outputs_zonalStatistic/treecover2000_30_'+country+'.csv')

    df_treecover2000_country30['forest_30'] = (
        df_treecover2000_country30['sum']*900)/10000

    df_treecover2000_country30 = df_treecover2000_country30.drop(
        ['sum', 'system:index', 'ADM1_CODE', 'ADM2_NAME', 'EXP2_YEAR',
         'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
         'ADM0_NAME', 'ADM0_CODE', 'Shape_Area'], axis=1)

    gpd_country_30 = pd.concat([
        gpd_country_30, df_treecover2000_country30.set_index('ADM2_CODE')],
        axis=1)

    gpd_country_30 = gpd_country_30.drop([
        'loss30_ha_2001', 'loss30_ha_2002',
        'loss30_ha_2003', 'loss30_ha_2004',
        'loss30_ha_2005', 'loss30_ha_2006',
        'loss30_ha_2007', 'loss30_ha_2008'
    ], axis=1)

    gpd_country_30.to_file(
        'gpkg/V1/thresholds.gpkg', layer=country+str(30), driver="GPKG")

    # return gpd_country_30

In [ ]:
listC = ['Argentina', 'Bolivia', 'Colombia', 'Ecuador', 'Guatemala', 
         'Honduras', 'Mexico', 'Peru', 'Venezuela']

for i in listC:
    TargetCountries30(i)

In [ ]:
# This is to split the state of Brazil into parts to reduce computational process.
lista1 = [i for i in range(6333, 8171)]
lista2 = [i for i in range(8171, 10008)]
lista3 = [i for i in range(10008, 11843)]

print(lista1[-1], lista2[-1], lista3[-1])

In [ ]:
# filter by target countries
Brazil = result2.filter(ee.Filter.eq(
  'ADM0_NAME', 'Brazil'));

StatesB1 = Brazil.filter(ee.Filter.inList('ADM2_CODE', lista1))
StatesB2 = Brazil.filter(ee.Filter.inList('ADM2_CODE', lista2))
StatesB3 = Brazil.filter(ee.Filter.inList('ADM2_CODE', lista3))

In [ ]:
Map.addLayer(StatesB2, tyleParams, 'TargetCountries', 1)
#Map.addLayerControl()
Map

In [ ]:
# This code piece generate one csv file by each loss year and country
country = StatesB1
for i in range(len(consecutive_numbers)):
    out_dir = os.path.join('outputs_zonalStatistic')
    out_countries_stats = os.path.join(
        out_dir, lossyear30[i] + 'StatesB1'+'.csv')
    
    geemap.zonal_statistics(
        deforestation30[i],
        country,
        out_countries_stats,
        statistics_type='SUM',
        scale=scale.getInfo()
    )
# -----------------------------------------------------------------------------
# Load csv layers generated in the previous step

filename = r'outputs_zonalStatistic/'
#nn = np.arange(2001,2019,1)
filenames = ['lossyear30_' + str(f) + 'StatesB1'+'.csv' for f in years]

df_DEF_StatesB130 = [pd.read_csv(filename + j) for j in filenames]

# -----------------------------------------------------------------------------
# Calculate area in ha from zonal statistic result
scale = scale.getInfo()
for i in range(len(df_DEF_StatesB130)):
    df_DEF_StatesB130[i]['loss30_ha_' + str(years[i])] = (
        (df_DEF_StatesB130[i]['sum']*(scale**2))/10000
    )

# -----------------------------------------------------------------------------
# download only once the FeatureCollection of the countries in shape format
# Uncomment the next lines to download it ONLY ONCE.
# This is to get a base in shp format to join csv's fiels.
out_dir = os.path.join('outputs_zonalStatistic/countriesSHP')
out_countries_stats = os.path.join(out_dir, 'StatesB1_50.shp')
    
geemap.zonal_statistics(
    deforestation30[0], 
    country, 
    out_countries_stats, 
    statistics_type='SUM', 
    scale=scale)

# -----------------------------------------------------------------------------
# Drop unnecesary columns
df_DEF_StatesB130_drop = []
for i in df_DEF_StatesB130:
    df_DEF_StatesB130_drop.append(i.drop([
        'sum', 'system:index', 'ADM1_CODE', 'EXP2_YEAR', 'ADM2_NAME', 
        'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
        'ADM0_NAME', 'ADM0_CODE', 'Shape_Area', 'areaSqKm'
    ], axis=1))
    
# -----------------------------------------------------------------------------   
gpd_StatesB1_30 = gpd.read_file(
    r'outputs_zonalStatistic/countriesSHP/StatesB1_30.shp')

gpd_StatesB1_30 = gpd_StatesB1_30.drop(['sum'], axis=1)

gpd_StatesB1_30 = gpd_StatesB1_30.set_index('ADM2_CODE')

for i in df_DEF_StatesB130_drop:
    gpd_StatesB1_30 = pd.concat([gpd_StatesB1_30, i.set_index('ADM2_CODE')],
                                axis=1)
# -----------------------------------------------------------------------------
out_dir = os.path.join('outputs_zonalStatistic')
out_countries_stats = os.path.join(out_dir,
                                   'treecover2000_30_StatesB1.csv')
    
geemap.zonal_statistics(
    treecover30,
    country,
    out_countries_stats, 
    statistics_type='SUM',
    scale=scale)

# -----------------------------------------------------------------------------
# concat with treecover 30%
df_treecover2000_StatesB130 = pd.read_csv(
    r'outputs_zonalStatistic/treecover2000_30_StatesB1.csv')

df_treecover2000_StatesB130['forest_30'] = (
    (df_treecover2000_StatesB130['sum']*(scale**2))/10000)

df_treecover2000_StatesB130 = df_treecover2000_StatesB130.drop(
    ['sum', 'system:index', 'ADM1_CODE', 'ADM2_NAME', 'EXP2_YEAR',
     'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
     'ADM0_NAME', 'ADM0_CODE', 'Shape_Area','areaSqKm'], 
    axis=1)

gpd_StatesB1_30 = pd.concat([
    gpd_StatesB1_30, 
    df_treecover2000_StatesB130.set_index('ADM2_CODE')], axis=1)

gpd_StatesB1_30 = gpd_StatesB1_30.drop(
    ['loss30_ha_2001', 'loss30_ha_2002', 
    'loss30_ha_2003', 'loss30_ha_2004',
    'loss30_ha_2005', 'loss30_ha_2006',
    'loss30_ha_2007', 'loss30_ha_2008'], axis=1)


gpd_StatesB1_30.to_file('gpkg/V1/thresholds.gpkg', layer='StatesB1_30',
                        driver="GPKG")

gpd_StatesB1_30[['areaSqKm','forest_30']]

In [ ]:
# filter for target countries
#country = result2.filter(ee.Filter.eq(
#    'ADM0_NAME', 'StatesB2'));
country = StatesB2
for i in range(len(consecutive_numbers)):
    out_dir = os.path.join('outputs_zonalStatistic')
    out_countries_stats = os.path.join(
        out_dir, lossyear30[i] + 'StatesB2'+'.csv')
    
    geemap.zonal_statistics(
        deforestation30[i],
        country,
        out_countries_stats,
        statistics_type='SUM',
        scale=scale
    )
# ----------------------------------------------------------------------------
# Load csv layers generated in the previous step

filename = r'outputs_zonalStatistic/'
#nn = np.arange(2001,2019,1)
filenames = ['lossyear30_' + str(f) + 'StatesB2'+'.csv' for f in years]

df_DEF_StatesB230 = [pd.read_csv(filename + j) for j in filenames]

# Calculate area in ha from result of the zonal statistic
scale = scale.getInfo()
for i in range(len(df_DEF_StatesB230)):
    df_DEF_StatesB230[i]['loss30_ha_' + str(years[i])] = (
        (df_DEF_StatesB230[i]['sum']*900)/10000
    )
    
# download for only one time the FeatureCollection of the countries in 
# shape format

# This is to get a base in shp format to join csv's fiels.
out_dir = os.path.join('outputs_zonalStatistic/countriesSHP')
out_countries_stats = os.path.join(out_dir, 'StatesB2_30.shp')
    
geemap.zonal_statistics(
    deforestation30[0], 
    country,
    out_countries_stats,
    statistics_type='SUM',
    scale=scale)
    
# Drop unnecesary columns
df_DEF_StatesB230_drop = []
for i in df_DEF_StatesB230:
    df_DEF_StatesB230_drop.append(i.drop([
        'sum', 'system:index', 'ADM1_CODE', 'EXP2_YEAR', 'ADM2_NAME', 
        'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
        'ADM0_NAME', 'ADM0_CODE', 'Shape_Area', 'areaSqKm'
    ], axis=1))
    
    
gpd_StatesB2_30 = gpd.read_file(
    r'outputs_zonalStatistic/countriesSHP/StatesB2_30.shp')
gpd_StatesB2_30 = gpd_StatesB2_30.drop(['sum'], axis=1)

gpd_StatesB2_30 = gpd_StatesB2_30.set_index('ADM2_CODE')

for i in df_DEF_StatesB230_drop:
    gpd_StatesB2_30 = pd.concat([gpd_StatesB2_30, i.set_index('ADM2_CODE')], 
                                axis=1)
# -----------------------------------------------------------------------------
out_dir = os.path.join('outputs_zonalStatistic')
out_countries_stats = os.path.join(out_dir,
                                   'treecover2000_30_StatesB2.csv')
    
geemap.zonal_statistics(
    treecover30,
    StatesB2,
    out_countries_stats,
    statistics_type='SUM',
    scale=scale)

# -----------------------------------------------------------------------------
# concat with treecover 30%
df_treecover2000_StatesB230 = pd.read_csv(
    r'outputs_zonalStatistic/treecover2000_30_StatesB2.csv')

df_treecover2000_StatesB230['forest_30'] = (
    df_treecover2000_StatesB230['sum']*(scale**2))/10000

df_treecover2000_StatesB230 = df_treecover2000_StatesB230.drop([
    'sum', 'system:index', 'ADM1_CODE', 'ADM2_NAME', 'EXP2_YEAR',
    'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
    'ADM0_NAME', 'ADM0_CODE', 'Shape_Area'], axis=1)

gpd_StatesB2_30 = pd.concat([
    gpd_StatesB2_30, df_treecover2000_StatesB230.set_index('ADM2_CODE')],
    axis=1)

gpd_StatesB2_30 = gpd_StatesB2_30.drop([
    'loss30_ha_2001', 'loss30_ha_2002', 
    'loss30_ha_2003', 'loss30_ha_2004',
    'loss30_ha_2005', 'loss30_ha_2006',
    'loss30_ha_2007', 'loss30_ha_2008'
], axis=1)


gpd_StatesB2_30.to_file('gpkg/V1/thresholds.gpkg', layer='StatesB2_30',
                        driver="GPKG")

In [ ]:
country = StatesB3
for i in range(len(consecutive_numbers)):
    out_dir = os.path.join('outputs_zonalStatistic')
    out_countries_stats = os.path.join(
        out_dir, lossyear30[i] + 'StatesB3'+'.csv')
    
    geemap.zonal_statistics(
        deforestation30[i],
        country,
        out_countries_stats,
        statistics_type='SUM',
        scale=scale
    )
#-------------------------------------------------------------------
# Load csv layers generated in the previous step

filename = r'outputs_zonalStatistic/'
#nn = np.arange(2001,2019,1)
filenames = ['lossyear30_' + str(f) + 'StatesB3'+'.csv' for f in years]

df_DEF_StatesB330 = [pd.read_csv(filename + j) for j in filenames]

# Calculate area in ha from result of the zonal statistic
scale = scale.getInfo()
for i in range(len(df_DEF_StatesB330)):
    df_DEF_StatesB330[i]['loss30_ha_' + str(years[i])] = (
        (df_DEF_StatesB330[i]['sum']*(scale**2))/10000
    )
    
# download for only one time the FeatureCollection of the countries in
# shape format

# This is to get a base in shp format to join csv's fiels.
out_dir = os.path.join('outputs_zonalStatistic/countriesSHP')
out_countries_stats = os.path.join(out_dir, 'StatesB3_50.shp')
    
geemap.zonal_statistics(
    deforestation30[0],
    country,
    out_countries_stats, 
    statistics_type='SUM', 
    scale=scale)
    
# Drop unnecesary columns
df_DEF_StatesB330_drop = []
for i in df_DEF_StatesB330:
    df_DEF_StatesB330_drop.append(i.drop([
        'sum', 'system:index', 'ADM1_CODE', 'EXP2_YEAR', 'ADM2_NAME', 
        'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
        'ADM0_NAME', 'ADM0_CODE', 'Shape_Area', 'areaSqKm'
    ], axis=1))
    
    
gpd_StatesB3_30 = gpd.read_file(
    r'outputs_zonalStatistic/countriesSHP/StatesB3_30.shp')
gpd_StatesB3_30 = gpd_StatesB3_30.drop(['sum'], axis=1)

gpd_StatesB3_30 = gpd_StatesB3_30.set_index('ADM2_CODE')

for i in df_DEF_StatesB330_drop:
    gpd_StatesB3_30 = pd.concat([gpd_StatesB3_30, i.set_index('ADM2_CODE')],
                                axis=1)
    
out_dir = os.path.join('outputs_zonalStatistic')
out_countries_stats = os.path.join(out_dir,
                                   'treecover2000_30_StatesB3.csv')
    
geemap.zonal_statistics(
    treecover30,
    country,
    out_countries_stats,
    statistics_type='SUM',
    scale=scale)

# -----------------------------------------------------------------------------
# concat with treecover 30%
df_treecover2000_StatesB330 = pd.read_csv(
    r'outputs_zonalStatistic/treecover2000_30_StatesB3.csv')

df_treecover2000_StatesB330['forest_30'] = (
    (df_treecover2000_StatesB330['sum']*(scale**2))/10000)

df_treecover2000_StatesB330 = df_treecover2000_StatesB330.drop(
    ['sum', 'system:index', 'ADM1_CODE', 'ADM2_NAME', 'EXP2_YEAR',
     'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
     'ADM0_NAME', 'ADM0_CODE', 'Shape_Area','areaSqKm'], 
    axis=1)

gpd_StatesB3_30 = pd.concat([
    gpd_StatesB3_30,
    df_treecover2000_StatesB330.set_index('ADM2_CODE')], axis=1)

gpd_StatesB3_30 = gpd_StatesB3_30.drop(
    ['loss30_ha_2001', 'loss30_ha_2002', 
    'loss30_ha_2003', 'loss30_ha_2004',
    'loss30_ha_2005', 'loss30_ha_2006',
    'loss30_ha_2007', 'loss30_ha_2008'], axis=1)


gpd_StatesB3_30.to_file('gpkg/V1/thresholds.gpkg', layer='StatesB3_30',
                        driver="GPKG")

gpd_StatesB3_30[['areaSqKm','forest_30']]

listCountries = ['Argentina', 'Bolivia', 'Brazil', 'Colombia', 'Ecuador',
                 'Guatemala', 'Honduras', 'Mexico', 'Peru', 'Venezuela']

## 3.2 For each loss year using treecover50%

In [ ]:
scale = lossImage.projection().nominalScale()
scale = scale.getInfo()
    
def TargetCountries50(country):
        # filter for target countries
    countryTarget = result2.filter(ee.Filter.eq(
      'ADM0_NAME', country))

    for i in range(len(consecutive_numbers)):
        out_dir = os.path.join('outputs_zonalStatistic')
        out_countries_stats = os.path.join(
            out_dir, lossyear50[i] + country+'.csv')

        geemap.zonal_statistics(
            deforestation50[i],
            countryTarget,
            out_countries_stats,
            statistics_type='SUM',
            scale=scale
        )
    #-------------------------------------------------------------------
    # Load csv layers generated in the previous step

    filename = r'outputs_zonalStatistic/'
    #nn = np.arange(2001,2019,1)
    filenames = ['lossyear50_' + str(f) + country + '.csv' for f in years]

    df_DEF_country50 = [pd.read_csv(filename + j) for j in filenames]

    # Calculate area in ha from result of the zonal statistic
    for i in range(len(df_DEF_country50)):
        df_DEF_country50[i]['loss50_ha_'+str(years[i])] = (
            df_DEF_country50[i]['sum']*(scale**2))/10000

    # download for only one time the FeatureCollection of the countries in 
    # shape format

    # This is to get a base in shp format to join csv's fiels.
    out_dir = os.path.join('outputs_zonalStatistic/countriesSHP')
    out_countries_stats = os.path.join(out_dir, country+'.shp')

    geemap.zonal_statistics(
        deforestation50[0], 
        countryTarget, 
        out_countries_stats, 
        statistics_type='SUM', 
        scale=scale)

    # Drop unnecesary columns
    df_DEF_country50_drop = []
    for i in df_DEF_country50:
        df_DEF_country50_drop.append(i.drop([
            'sum', 'system:index', 'ADM1_CODE', 'EXP2_YEAR', 'ADM2_NAME', 
            'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
            'ADM0_NAME', 'ADM0_CODE', 'Shape_Area', 'areaSqKm'
        ], axis=1))


    gpd_country_50 = gpd.read_file(
        r'outputs_zonalStatistic/countriesSHP/'+country+'.shp')
    gpd_country_50 = gpd_country_50.drop(['sum'], axis=1)

    gpd_country_50 = gpd_country_50.set_index('ADM2_CODE')

    for i in df_DEF_country50_drop:
        gpd_country_50 = pd.concat([gpd_country_50, i.set_index('ADM2_CODE')],
                                   axis=1)
        
        
    out_dir = os.path.join('outputs_zonalStatistic')
    out_countries_stats = os.path.join(out_dir,
                                       'treecover2000_50_'+country+'.csv')

    geemap.zonal_statistics(
        treecover50, 
        countryTarget, 
        out_countries_stats, 
        statistics_type='SUM',
        scale=scale)

    #---------------------------------------------------------------------
    # concat treecover 50%
    df_treecover2000_country50 = pd.read_csv(
        r'outputs_zonalStatistic/treecover2000_50_'+country+'.csv')

    df_treecover2000_country50['forest_50'] = (
        df_treecover2000_country50['sum']*(scale**2))/10000

    df_treecover2000_country50 = df_treecover2000_country50.drop(
        ['sum', 'system:index', 'ADM1_CODE', 'ADM2_NAME', 'EXP2_YEAR',
         'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR', 
         'ADM0_NAME', 'ADM0_CODE', 'Shape_Area'], axis=1)

    gpd_country_50 = pd.concat([
        gpd_country_50, df_treecover2000_country50.set_index('ADM2_CODE')],
        axis=1)

    gpd_country_50 = gpd_country_50.drop([
        'loss50_ha_2001', 'loss50_ha_2002', 
        'loss50_ha_2003', 'loss50_ha_2004',
        'loss50_ha_2005', 'loss50_ha_2006',
        'loss50_ha_2007', 'loss50_ha_2008'
    ], axis=1)


    gpd_country_50.to_file('gpkg/V1/thresholds.gpkg', layer=country,
                           driver="GPKG")
    
    return gpd_country_50

In [ ]:
listC = ['Bolivia', 'Colombia', 'Ecuador', 'Guatemala', 'Honduras', 'Peru',
         'Venezuela']

for i in listC:
    TargetCountries50(i)

In [ ]:
# This code piece generate one csv file by each loss year and country
country = StatesB1
for i in range(len(consecutive_numbers)):
    out_dir = os.path.join('outputs_zonalStatistic')
    out_countries_stats = os.path.join(
        out_dir, lossyear50[i] + 'StatesB1'+'.csv')
    
    geemap.zonal_statistics(
        deforestation50[i],
        country,
        out_countries_stats,
        statistics_type='SUM',
        scale=scale.getInfo()
    )
# -----------------------------------------------------------------------------
# Load csv layers generated in the previous step

filename = r'outputs_zonalStatistic/'
#nn = np.arange(2001,2019,1)
filenames = ['lossyear50_' + str(f) + 'StatesB1'+'.csv' for f in years]

df_DEF_StatesB150 = [pd.read_csv(filename + j) for j in filenames]

# -----------------------------------------------------------------------------
# Calculate area in ha from zonal statistic result
scale = scale.getInfo()
for i in range(len(df_DEF_StatesB150)):
    df_DEF_StatesB150[i]['loss50_ha_' + str(years[i])] = (
        (df_DEF_StatesB150[i]['sum']*(scale**2))/10000
    )

# -----------------------------------------------------------------------------
# download only once the FeatureCollection of the countries in shape format

# This is to get a base in shp format to join csv's fiels.
out_dir = os.path.join('outputs_zonalStatistic/countriesSHP')
out_countries_stats = os.path.join(out_dir, 'StatesB1_50.shp')
    
geemap.zonal_statistics(
    deforestation50[0], 
    country, 
    out_countries_stats, 
    statistics_type='SUM', 
    scale=scale)

# -----------------------------------------------------------------------------
# Drop unnecesary columns
df_DEF_StatesB150_drop = []
for i in df_DEF_StatesB150:
    df_DEF_StatesB150_drop.append(i.drop([
        'sum', 'system:index', 'ADM1_CODE', 'EXP2_YEAR', 'ADM2_NAME', 
        'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
        'ADM0_NAME', 'ADM0_CODE', 'Shape_Area', 'areaSqKm'
    ], axis=1))
    
# -----------------------------------------------------------------------------   
gpd_StatesB1_50 = gpd.read_file(
    r'outputs_zonalStatistic/countriesSHP/StatesB1_50.shp')

gpd_StatesB1_50 = gpd_StatesB1_50.drop(['sum'], axis=1)

gpd_StatesB1_50 = gpd_StatesB1_50.set_index('ADM2_CODE')

for i in df_DEF_StatesB150_drop:
    gpd_StatesB1_50 = pd.concat([gpd_StatesB1_50, i.set_index('ADM2_CODE')],
                                axis=1)
# -----------------------------------------------------------------------------
out_dir = os.path.join('outputs_zonalStatistic')
out_countries_stats = os.path.join(out_dir,
                                   'treecover2000_50_StatesB1.csv')
    
geemap.zonal_statistics(
    treecover50, 
    country, 
    out_countries_stats, 
    statistics_type='SUM',
    scale=scale)

# -----------------------------------------------------------------------------
# concat with treecover 50%
df_treecover2000_StatesB150 = pd.read_csv(
    r'outputs_zonalStatistic/treecover2000_50_StatesB1.csv')

df_treecover2000_StatesB150['forest_50'] = (
    (df_treecover2000_StatesB150['sum']*(scale**2))/10000)

df_treecover2000_StatesB150 = df_treecover2000_StatesB150.drop(
    ['sum', 'system:index', 'ADM1_CODE', 'ADM2_NAME', 'EXP2_YEAR',
     'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
     'ADM0_NAME', 'ADM0_CODE', 'Shape_Area','areaSqKm'], 
    axis=1)

gpd_StatesB1_50 = pd.concat([
    gpd_StatesB1_50, 
    df_treecover2000_StatesB150.set_index('ADM2_CODE')], axis=1)

gpd_StatesB1_50 = gpd_StatesB1_50.drop(
    ['loss50_ha_2001', 'loss50_ha_2002', 
    'loss50_ha_2003', 'loss50_ha_2004',
    'loss50_ha_2005', 'loss50_ha_2006',
    'loss50_ha_2007', 'loss50_ha_2008'], axis=1)


gpd_StatesB1_50.to_file('gpkg/V1/thresholds.gpkg', layer='StatesB1_50',
                        driver="GPKG")

gpd_StatesB1_50[['areaSqKm','forest_50']]

In [ ]:
country = StatesB2
for i in range(len(consecutive_numbers)):
    out_dir = os.path.join('outputs_zonalStatistic')
    out_countries_stats = os.path.join(
        out_dir, lossyear50[i] + 'StatesB2'+'.csv')
    
    geemap.zonal_statistics(
        deforestation50[i],
        country,
        out_countries_stats,
        statistics_type='SUM',
        scale=scale
    )
#-------------------------------------------------------------------
# Load csv layers generated in the previous step

filename = r'outputs_zonalStatistic/'
#nn = np.arange(2001,2019,1)
filenames = ['lossyear50_' + str(f) + 'StatesB2'+'.csv' for f in years]

df_DEF_StatesB250 = [pd.read_csv(filename + j) for j in filenames]

# Calculate area in ha from result of the zonal statistic
scale = scale.getInfo()
for i in range(len(df_DEF_StatesB250)):
    df_DEF_StatesB250[i]['loss50_ha_' + str(years[i])] = (
        (df_DEF_StatesB250[i]['sum']*900)/10000
    )
    
# download for only one time the FeatureCollection of the countries in
# shape format

# This is to get a base in shp format to join csv's fiels.
out_dir = os.path.join('outputs_zonalStatistic/countriesSHP')
out_countries_stats = os.path.join(out_dir, 'StatesB2_50.shp')
    
geemap.zonal_statistics(
    deforestation50[0], 
    country, 
    out_countries_stats, 
    statistics_type='SUM', 
    scale=scale)
    
# Drop unnecesary columns
df_DEF_StatesB250_drop = []
for i in df_DEF_StatesB250:
    df_DEF_StatesB250_drop.append(i.drop([
        'sum', 'system:index', 'ADM1_CODE', 'EXP2_YEAR', 'ADM2_NAME', 
        'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
        'ADM0_NAME', 'ADM0_CODE', 'Shape_Area', 'areaSqKm'
    ], axis=1))
    
    
gpd_StatesB2_50 = gpd.read_file(
    r'outputs_zonalStatistic/countriesSHP/StatesB2_50.shp')
gpd_StatesB2_50 = gpd_StatesB2_50.drop(['sum'], axis=1)

gpd_StatesB2_50 = gpd_StatesB2_50.set_index('ADM2_CODE')

for i in df_DEF_StatesB250_drop:
    gpd_StatesB2_50 = pd.concat([gpd_StatesB2_50, i.set_index('ADM2_CODE')], 
                                axis=1)
# -----------------------------------------------------------------------------
out_dir = os.path.join('outputs_zonalStatistic')
out_countries_stats = os.path.join(out_dir,
                                   'treecover2000_50_StatesB2.csv')
    
geemap.zonal_statistics(
    treecover50, 
    StatesB2, 
    out_countries_stats, 
    statistics_type='SUM',
    scale=scale)

# -----------------------------------------------------------------------------
# concat with treecover 50%
df_treecover2000_StatesB250 = pd.read_csv(
    r'outputs_zonalStatistic/treecover2000_50_StatesB2.csv')

df_treecover2000_StatesB250['forest_50'] = (
    df_treecover2000_StatesB250['sum']*(scale**2))/10000

df_treecover2000_StatesB250 = df_treecover2000_StatesB250.drop([
    'sum', 'system:index', 'ADM1_CODE', 'ADM2_NAME', 'EXP2_YEAR',
    'ADM1_NAME', 'DISP_AREA', 'Shape_Leng', 'STATUS', 'STR2_YEAR',
    'ADM0_NAME', 'ADM0_CODE', 'Shape_Area'], axis=1)

gpd_StatesB2_50 = pd.concat([
    gpd_StatesB2_50, df_treecover2000_StatesB250.set_index('ADM2_CODE')],
    axis=1)

gpd_StatesB2_50 = gpd_StatesB2_50.drop([
    'loss50_ha_2001', 'loss50_ha_2002', 
    'loss50_ha_2003', 'loss50_ha_2004',
    'loss50_ha_2005', 'loss50_ha_2006',
    'loss50_ha_2007', 'loss50_ha_2008'
], axis=1)


gpd_StatesB2_50.to_file('gpkg/V1/thresholds.gpkg', layer='StatesB2_50',
                        driver="GPKG")

In [ ]:
%pycodestyle_off